<a href="https://colab.research.google.com/github/Aleeg10/Quant/blob/main/MeanRevers_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: make it intercative and upgrade visualization

# ================================
# 5. Interactive Visualization
# ================================
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

def plot_strategy(S0, mu, sigma, window, entry_z, exit_z):
    np.random.seed(42) # Use a fixed seed for consistent comparison with sliders

    T = 252 * 2  # 2 years
    dt = 1/252   # Daily time step

    # Simulate GBM
    time = np.linspace(0, T * dt, T)
    W = np.random.standard_normal(size=T)
    W = np.cumsum(W) * np.sqrt(dt)  # Brownian motion
    S = S0 * np.exp((mu - 0.5 * sigma ** 2) * time + sigma * W)

    # Mean-Reversion Strategy (Z-score)
    df = pd.DataFrame({'Price': S})
    df['Rolling_Mean'] = df['Price'].rolling(window=window).mean()
    df['Rolling_Std'] = df['Price'].rolling(window=window).std()
    df['Z_score'] = (df['Price'] - df['Rolling_Mean']) / df['Rolling_Std']

    # Entry/exit logic
    df['Position'] = 0
    df.loc[df['Z_score'] > entry_z, 'Position'] = -1  # Short when too high
    df.loc[df['Z_score'] < -entry_z, 'Position'] = 1  # Long when too low

    # Add exit condition using exit_z (optional, can keep simple entry)
    # For simplicity, we'll keep the entry-based exit for now, exiting when Z-score crosses back below entry_z
    # A more sophisticated exit would involve a different threshold
    # Let's refine the position based on both entry and exit z-scores
    df['Position'] = 0
    # Go short if Z_score > entry_z
    df.loc[df['Z_score'] > entry_z, 'Position'] = -1
    # Go long if Z_score < -entry_z
    df.loc[df['Z_score'] < -entry_z, 'Position'] = 1

    # Implement a simple exit strategy: exit position when Z-score crosses zero or a small threshold
    # We need to track the previous position to know if we were in a trade
    df['Prev_Position'] = df['Position'].shift(1).fillna(0)

    # If prev position was 1 (long) and Z_score crosses above -exit_z, set current position to 0
    df.loc[(df['Prev_Position'] == 1) & (df['Z_score'] >= -exit_z), 'Position'] = 0
    # If prev position was -1 (short) and Z_score crosses below exit_z, set current position to 0
    df.loc[(df['Prev_Position'] == -1) & (df['Z_score'] <= exit_z), 'Position'] = 0

    # Handle cases where Z_score stays beyond entry_z or -entry_z.
    # If in a long position (Prev_Position == 1) and Z_score is still < -entry_z, maintain long position.
    df.loc[(df['Prev_Position'] == 1) & (df['Z_score'] < -entry_z), 'Position'] = 1
    # If in a short position (Prev_Position == -1) and Z_score is still > entry_z, maintain short position.
    df.loc[(df['Prev_Position'] == -1) & (df['Z_score'] > entry_z), 'Position'] = -1


    # This basic implementation might still flip positions rapidly.
    # For a more robust strategy, you would need to ensure positions are held until the exit condition is met.
    # A common way is to fill forward the position until an exit signal occurs.

    # Let's try a simpler approach: use shift to ensure positions are held
    # Start with initial position 0
    df['Position'] = 0

    # Identify entry signals
    long_entry = df['Z_score'] < -entry_z
    short_entry = df['Z_score'] > entry_z

    # Identify exit signals (crossing back towards zero)
    long_exit = df['Z_score'] >= -exit_z # Exit long when Z-score rises to or above -exit_z
    short_exit = df['Z_score'] <= exit_z # Exit short when Z-score falls to or below exit_z

    # Determine position based on signals
    position = pd.Series(index=df.index, data=0.0)

    # Fill position based on signals: long until long_exit, short until short_exit
    # This requires a loop or forward fill logic, which is less efficient in pandas without loops.
    # A more idiomatic pandas approach for state tracking is needed.

    # Let's try a state machine approach
    state = 0 # 0: flat, 1: long, -1: short
    positions = []

    for i in range(len(df)):
        current_z = df['Z_score'].iloc[i]

        if state == 0: # Currently flat
            if current_z < -entry_z:
                state = 1 # Enter long position
            elif current_z > entry_z:
                state = -1 # Enter short position
        elif state == 1: # Currently long
            if current_z >= -exit_z:
                state = 0 # Exit long position
        elif state == -1: # Currently short
            if current_z <= exit_z:
                state = 0 # Exit short position

        positions.append(state)

    df['Position'] = positions

    # Ensure position is applied on the next day after the signal
    df['Position'] = df['Position'].shift(1).fillna(0)


    # Returns
    df['Returns'] = df['Price'].pct_change().fillna(0)
    df['Strategy_Returns'] = df['Returns'] * df['Position']
    df['Cumulative_Strategy'] = (1 + df['Strategy_Returns']).cumprod()
    df['Cumulative_Price'] = (1 + df['Returns']).cumprod()

    # Plotting
    plt.figure(figsize=(14, 8)) # Increased figure size
    plt.plot(df['Cumulative_Price'], label='Buy & Hold', linewidth=2) # Thicker line
    plt.plot(df['Cumulative_Strategy'], label='Mean-Reversion Strategy', linewidth=2) # Thicker line
    plt.title('GBM Simulation with Mean-Reversion Strategy', fontsize=16) # Larger title
    plt.xlabel('Days', fontsize=12) # Larger labels
    plt.ylabel('Cumulative Returns', fontsize=12) # Larger labels
    plt.legend(fontsize=12) # Larger legend text
    plt.grid(True, linestyle='--', alpha=0.6) # Dotted grid
    plt.yscale('linear') # Ensure linear scale unless log is explicitly needed

    # Add a secondary y-axis for Z-score (optional, but can be helpful)
    fig, ax1 = plt.subplots(figsize=(14, 8))

    ax1.plot(df.index, df['Cumulative_Price'], label='Buy & Hold', linewidth=2)
    ax1.plot(df.index, df['Cumulative_Strategy'], label='Mean-Reversion Strategy', linewidth=2)
    ax1.set_xlabel('Days', fontsize=12)
    ax1.set_ylabel('Cumulative Returns', fontsize=12)
    ax1.tick_params(axis='y')
    ax1.set_title('GBM Simulation with Mean-Reversion Strategy and Z-score', fontsize=16)
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.legend(loc='upper left', fontsize=12)

    ax2 = ax1.twinx() # Create a second y-axis sharing the same x-axis
    ax2.plot(df.index, df['Z_score'], label='Z-score', color='gray', alpha=0.5, linestyle='-') # Plot Z-score
    ax2.axhline(entry_z, color='red', linestyle='--', label=f'Entry Short Threshold ({entry_z:.1f})')
    ax2.axhline(-entry_z, color='green', linestyle='--', label=f'Entry Long Threshold ({-entry_z:.1f})')
    ax2.axhline(exit_z, color='orange', linestyle=':', label=f'Exit Short Threshold ({exit_z:.1f})')
    ax2.axhline(-exit_z, color='purple', linestyle=':', label=f'Exit Long Threshold ({-exit_z:.1f})')

    ax2.set_ylabel('Z-score', color='gray', fontsize=12)
    ax2.tick_params(axis='y', labelcolor='gray')
    ax2.legend(loc='upper right', fontsize=10)

    fig.tight_layout() # Adjust layout to prevent overlapping
    plt.show()


# Create interactive widgets
interact(plot_strategy,
         S0=widgets.FloatSlider(min=50, max=200, step=5, value=100, description='Initial Price:'),
         mu=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1, description='Expected Return (%):'),
         sigma=widgets.FloatSlider(min=0.05, max=0.4, step=0.01, value=0.2, description='Volatility (%):'),
         window=widgets.IntSlider(min=10, max=100, step=5, value=20, description='Rolling Window:'),
         entry_z=widgets.FloatSlider(min=0.5, max=3.0, step=0.1, value=1.0, description='Entry Z-score:'),
         exit_z=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Exit Z-score:'));



interactive(children=(FloatSlider(value=100.0, description='Initial Price:', max=200.0, min=50.0, step=5.0), F…